# Using _egobox_ optimizer _Egor_

## Installation

In [1]:
# %pip install egobox

We import _egobox_ as _egx_ for short.

In [2]:
import numpy as np
import egobox as egx

You may setup the logging level to get optimization progress during the execution

In [3]:
# To display optimization information (none by default)
import logging
logging.basicConfig(level=logging.INFO)

## Example 1 : Continuous optimization

### Test functions

In [4]:
xspecs_xsinx = [[0., 25.]]
n_cstr_xsinx = 0

def xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    return y

In [5]:
xspecs_g24 = [[0., 3.], [0., 4.]]
n_cstr_g24 = 2

# Objective
def G24(point):
    """
    Function g24
    1 global optimum y_opt = -5.5080 at x_opt =(2.3295, 3.1785)
    """
    p = np.atleast_2d(point)
    return - p[:, 0] - p[:, 1]

# Constraints < 0
def G24_c1(point):
    p = np.atleast_2d(point)
    return (- 2.0 * p[:, 0] ** 4.0
            + 8.0 * p[:, 0] ** 3.0 
            - 8.0 * p[:, 0] ** 2.0 
            + p[:, 1] - 2.0)

def G24_c2(point):
    p = np.atleast_2d(point)
    return (-4.0 * p[:, 0] ** 4.0
            + 32.0 * p[:, 0] ** 3.0
            - 88.0 * p[:, 0] ** 2.0
            + 96.0 * p[:, 0]
            + p[:, 1] - 36.0)

# Grouped evaluation
def g24(point):
    p = np.atleast_2d(point)
    return np.array([G24(p), G24_c1(p), G24_c2(p)]).T


### Continuous optimization

In [6]:
egor = egx.Egor(xspecs_xsinx, n_cstr=n_cstr_xsinx)   # see help(egor) for options

In [7]:
egor = egx.Egor(xspecs_g24, 
                n_doe=10, 
                n_cstr=n_cstr_g24, 
                cstr_tol=[1e-3, 1e-3],
                infill_strategy=egx.InfillStrategy.WB2,
                target=-5.5,
                trego=False,
            )   

In [8]:
res = egor.minimize(g24, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

INFO:egobox_ego.egor:EgorConfig { max_iters: 30, n_start: 20, n_doe: 10, n_cstr: 2, cstr_tol: Some([0.001, 0.001], shape=[2], strides=[1], layout=CFcf (0xf), const ndim=1), doe: None, q_ei: KrigingBeliever, q_optmod: 1, q_points: 1, gp: GpConfig { regression_spec: RegressionSpec(CONSTANT), correlation_spec: CorrelationSpec(SQUAREDEXPONENTIAL), kpls_dim: None, n_clusters: Fixed { nb: 1 }, recombination: Hard, theta_tuning: Full { init: [0.1], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1, bounds: [(0.001, 20.0)], shape=[1], strides=[1], layout=CFcf (0xf), const ndim=1 }, n_start: 10, max_eval: 50 }, infill_criterion: WB2, infill_optimizer: Cobyla, target: -5.5, outdir: None, warm_start: false, hot_start: Disabled, xtypes: [Cont(0.0, 3.0), Cont(0.0, 4.0)], seed: None, trego: TregoConfig { activated: false, n_local_steps: 4, d: (1e-6, 1.0), beta: 0.9, gamma: 1.1111111111111112, sigma0: 0.1 }, coego: CoegoConfig { activated: false, n_coop: 5 }, cstr_infill: false, cstr_strategy: 

Optimization f=[-5.50812317e+00 -2.38941208e-04  2.50732758e-04] at [2.32955826 3.17856492]
Optimization history: 
Inputs = [[0.49068648 2.55232776]
 [2.52374908 2.92941602]
 [2.8315787  0.23572418]
 [1.29534423 1.84673401]
 [0.72083677 1.48353299]
 [2.34610675 0.89706217]
 [0.0348933  0.41276329]
 [1.62777533 3.81556788]
 [1.91403138 2.07830413]
 [0.94259975 3.51671903]
 [2.3658419  3.14716742]
 [2.3297988  3.18306562]
 [2.32973352 3.17851078]
 [2.32955826 3.17856492]]
Outputs = [[-3.04301425e+00 -5.44648184e-01 -3.98108733e+00]
 [-5.45316510e+00 -2.56495404e+00  8.22930123e-01]
 [-3.06730287e+00 -1.28533166e+01 -1.44907736e-01]
 [-3.14207824e+00 -1.81957065e+00  8.32845107e-01]
 [-2.20436976e+00 -2.21688540e+00 -1.35766857e-01]
 [-3.24316892e+00 -2.42163682e+00 -2.20201723e+00]
 [-4.47656597e-01 -1.59664014e+00 -3.23432696e+01]
 [-5.44334321e+00  1.08134278e+00  8.47191783e-01]
 [-3.99233551e+00  2.41529371e-02 -1.86278953e+00]
 [-4.45931879e+00 -4.70123523e-01  3.46093305e+00]
 [-5.

### Egor as a service: ask-and-tell interface

When the user needs to be in control of the optimization loop, `Egor` can be used as a service. 

For instance with the `xsinx` objective function, we can do:

In [9]:
xlimits = [[0.0, 25.0]]
egor = egx.Egor(xlimits, seed=42) 

# initial doe
x_doe = egx.lhs(xlimits, 3, seed=42)
y_doe = xsinx(x_doe)
for _ in range(10): # run for 10 iterations
    x = egor.suggest(x_doe, y_doe)  # ask for best location
    x_doe = np.concatenate((x_doe, x))
    y_doe = np.concatenate((y_doe, xsinx(x))) 
res = egor.get_result(x_doe, y_doe)
print(f"Optimization f={res.y_opt} at {res.x_opt}")

PanicException: Error in xspecs conversion: PyErr { type: <class 'AttributeError'>, value: AttributeError("'list' object has no attribute 'xtype'"), traceback: None }

## Example 2 : Mixed-integer optimization

### Test function

In [ ]:
xspecs_mixint_xsinx = [egx.XSpec(egx.XType.INT, [0, 25])]
n_cstr_mixint_xsinx = 0

def mixint_xsinx(x: np.ndarray) -> np.ndarray:
    x = np.atleast_2d(x)
    if (np.abs(np.linalg.norm(np.floor(x))-np.linalg.norm(x))< 1e-8):
        y = (x - 3.5) * np.sin((x - 3.5) / (np.pi))
    else:
        raise ValueError(f"Bad input: mixint_xsinx accepts integer only, got {x}")
    return y

### Mixed-integer optimization with _Egor_

In [ ]:
egor = egx.Egor(xspecs_mixint_xsinx, 
                     n_doe=3, 
                     infill_strategy=egx.InfillStrategy.EI,
                     target=-15.12,
                    )  # see help(egor) for options
res = egor.minimize(mixint_xsinx, max_iters=30)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

## Example 3 : More mixed-integer optimization

In the following example we see we can have other special integer type cases, where a component of x can take one value out of a list of ordered values (ORD type) or being like an enum value (ENUM type). Those types differ by the processing related to the continuous relaxation made behind the scene:
* For INT type, resulting float is rounded to the closest int value,
* For ORD type, resulting float is cast to closest value among the given valid ones,
* For ENUM type, one hot encoding is performed to give the resulting value.  

### Test function

In [ ]:
# Objective function which takes [FLOAT, ENUM1, ENUM2, ORD] as input
# Note that ENUM values are passed as indice value eg either 0, 1 or 2 for a 3-sized enum  
def mixobj(X):
    # float
    x1 = X[:, 0]
    #  ENUM 1
    c1 = X[:, 1]
    x2 = c1 == 0
    x3 = c1 == 1
    x4 = c1 == 2
    #  ENUM 2
    c2 = X[:, 2]
    x5 = c2 == 0
    x6 = c2 == 1
    # int
    i = X[:, 3]

    y = (x2 + 2 * x3 + 3 * x4) * x5 * x1 + (x2 + 2 * x3 + 3 * x4) * x6 * 0.95 * x1 + i
    return y.reshape(-1, 1)

### Mixed-integer optimization with _Egor_

In [ ]:
xtypes = [
    egx.XSpec(egx.XType.FLOAT, [-5.0, 5.0]),
    egx.XSpec(egx.XType.ENUM, tags=["blue", "red", "green"]),
    egx.XSpec(egx.XType.ENUM, xlimits=[2]),
    egx.XSpec(egx.XType.ORD, [0, 2, 3]),
]
egor = egx.Egor(xtypes, seed=42)
res = egor.minimize(mixobj, max_iters=10)
print(f"Optimization f={res.y_opt} at {res.x_opt}")
print("Optimization history: ")
print(f"Inputs = {res.x_doe}")
print(f"Outputs = {res.y_doe}")

Note that `x_opt` result contains indices for corresponding optional tags list hence the second component should be read as 0="red", 1="green", 2="blue", while the third component was unamed 0 correspond to first enum value and 1 to the second one.

## Usage

In [ ]:
help(egor)

In [ ]:
help(egx.GpConfig)